In [1]:
#creation du noeud: état intial du plateau
import pygame
import time

# Initialisation de Pygame
pygame.init()

# Variables globales pour compter les nœuds explorés
nodes_explored_minimax = 0
nodes_explored_alphabeta = 0

class Noeud:
    def __init__(self, grille=None, joueur_actuel=1, parent=None):

        if grille is None:
            # Position des pions selon l'image fournie
            self.grille = [
                [1, 1, 1],  # Joueur 1 (O) en haut aux coins
                [0, 0, -1],  # Ligne vide au centre
                [-1, 0, -1]  # Joueur 2 (X) en bas aux coins
            ]
        else:
            self.grille = [ligne[:] for ligne in grille]  # Copie du plateau

        self.joueur_actuel = joueur_actuel
        self.parent = parent
        self.enfants = []

    def afficher_grille(self):
        """Affiche le plateau sous forme lisible."""
        symbols = {0: ".", 1: "X", -1: "O"}
        for ligne in self.grille:
            print(" ".join(symbols[cell] for cell in ligne))
        print()
        
    def deplacements_possibles(self, x, y):
        """Retourne une liste des mouvements valides pour un pion donné."""
        directions = [
            (0, 1),  # Déplacement horizontal à droite
            (0, -1), # Déplacement horizontal à gauche
            (1, 0),  # Déplacement vertical vers le bas
            (-1, 0), # Déplacement vertical vers le haut
        ]

        # Liste des coins où les pions peuvent se déplacer en diagonale
        croix_intersections = [(0, 0), (0, 2), (2, 0), (2, 2)]  # Coins de la grille
        mouvements = []

        # Ajout des diagonales seulement pour les coins
        if (x, y) in croix_intersections:
            directions += [
                (1, 1), # Diagonale bas-gauche à haut-droit
                (1, -1),  # Diagonale haut-droit à bas-gauche
                (-1, 1), # Diagonale bas-droit à haut-gauche
                (-1, -1),  # Diagonale haut-gauche à bas-droit
            ]

        # Vérification des déplacements
        for dx, dy in directions:
            nx, ny = x + dx, y + dy

            # Vérifier que la nouvelle position est dans la grille et vide
            if 0 <= nx < 3 and 0 <= ny < 3 and self.grille[nx][ny] == 0:
                mouvements.append((nx, ny))

        return mouvements

    #question 1: methode get_successor()
    def get_successor(self):
        """Génère tous les états possibles après un coup."""
        successors = []
        for i in range(3):
            for j in range(3):
                if self.grille[i][j] == self.joueur_actuel:  # Si c'est un pion du joueur actuel
                    for nx, ny in self.deplacements_possibles(i, j):
                        nouvelle_grille = [ligne[:] for ligne in self.grille]
                        nouvelle_grille[i][j] = 0  # Enlever le pion de sa position actuelle
                        nouvelle_grille[nx][ny] = self.joueur_actuel  # Placer à la nouvelle position
                        successors.append(Noeud(nouvelle_grille, -self.joueur_actuel, self))
        self.successors = successors
        return successors

    #question 2: methode check_winner()
    def check_winner(self):
        """Vérifie si le joueur actuel a gagné."""
        for i in range(3):
            # Vérification horizontale et verticale
            if abs(sum(self.grille[i])) == 3 or abs(sum(row[i] for row in self.grille)) == 3:
                return True
        # Vérification diagonale
        if abs(self.grille[0][0] + self.grille[1][1] + self.grille[2][2]) == 3:
            return True
        if abs(self.grille[0][2] + self.grille[1][1] + self.grille[2][0]) == 3:
            return True
        return False

    def is_terminal(self):
        # Le jeu est terminé s'il y a un gagnant ou si le plateau est plein
        return self.check_winner() is not None or self.is_full()
    
    def eval(self, us):
        """Évalue l'état actuel du jeu pour le joueur 'us'."""
        if self.check_winner():
            # Si le joueur actuel a gagné, renvoie une valeur positive ou négative selon le joueur
            return 1 if self.joueur_actuel == us else -1
        # Si c'est un match nul, renvoie 0
        elif all(self.grille[i][j] != 0 for i in range(3) for j in range(3)):
            return 0  # Match nul
        return 0  # Jeu en cours (nous retournons une évaluation neutre)


#question 3: implémentation de mimimax()
def minimax(node, depth, us='X'):
    global nodes_explored_minimax
    nodes_explored_minimax += 1  # Incrémenter le compteur
    
    if depth == 0 or node.check_winner():
        return (node.eval(us), None)

    if node.joueur_actuel == us:
        maxEval = -1000
        best_move = None
        for child in node.get_successor():
            eval, _ = minimax(child, depth - 1, us)
            if eval > maxEval:
                maxEval = eval
                best_move = child
        return (maxEval, best_move)                
    else:
        minEval = 1000
        best_move = None
        for child in node.get_successor():
            eval, _ = minimax(child, depth - 1, us)
            if eval < minEval:
                minEval = eval
                best_move = child
        return (minEval, best_move)
    
#question 4: implémentation d'alphabeta()    
def alphabeta(node, depth, alpha, beta, us='X'):
    global nodes_explored_alphabeta
    nodes_explored_alphabeta += 1  # Incrémenter le compteur
    
    if depth == 0 or node.is_terminal():
        return (node.eval(us), None)
    if node.joueur_actuel == us:
        maxEval = -1000
        best_move = None
        for child in node.get_successor():
            eval, _ = alphabeta(child, depth - 1, alpha, beta, us)
            if eval > maxEval:
                maxEval = eval
                best_move = child
            alpha = max(alpha, eval)
            if beta <= alpha:
                break
        return (maxEval, best_move)
    else:
        minEval = 1000
        best_move = None
        for child in node.get_successor():
            eval, _ = alphabeta(child, depth - 1, alpha, beta, us)
            if eval < minEval:
                minEval = eval
                best_move = child
            beta = min(beta, eval)
            if beta <= alpha:
                break
        return (minEval, best_move)

#question 5  
# Fonction d'expérimentation
def comparer_performance():
    global nodes_explored_minimax, nodes_explored_alphabeta

    for depth in range(1, 6):  # Tester de profondeur 1 à 5
        noeud_initial = Noeud()

        # Réinitialiser les compteurs
        nodes_explored_minimax = 0
        nodes_explored_alphabeta = 0

        # Mesurer le temps d'exécution du Minimax
        start_time = time.time()
        minimax(noeud_initial, depth, us=1)
        minimax_time = time.time() - start_time

        # Mesurer le temps d'exécution de l'Alpha-Bêta
        start_time = time.time()
        alphabeta(noeud_initial, depth, float('-inf'), float('inf'), us=1)
        alphabeta_time = time.time() - start_time

        # Affichage des résultats
        print(f"Profondeur : {depth}")
        print(f"  Minimax -> Temps : {minimax_time:.4f}s, Nœuds explorés : {nodes_explored_minimax}")
        print(f"  Alpha-Bêta -> Temps : {alphabeta_time:.4f}s, Nœuds explorés : {nodes_explored_alphabeta}")
        print("-" * 40)

# Constantes
LARGEUR_FENETRE = 300
HAUTEUR_FENETRE = 300
TAILLE_CASE = 100
COULEUR_FOND = (255, 255, 255)
COULEUR_LIGNE = (255, 255, 255)
Noir = (0, 0, 0)
Gris = (150, 150, 150)

# Création de la fenêtre
screen = pygame.display.set_mode((LARGEUR_FENETRE, HAUTEUR_FENETRE))
pygame.display.set_caption("Fanorona 3x3")

# Charger l'image de fond
image_fond = pygame.image.load("C:/Users/user/F1.jpg")
image_fond = pygame.transform.scale(image_fond, (LARGEUR_FENETRE, HAUTEUR_FENETRE))

# Instance de la classe Noeud pour l'état du jeu
noeud_actuel = Noeud()

# Joueur actuel
joueur_actuel = 1

# Listes pour stocker les positions des clics et les coups
clics = []
coups_p1 = 0
coups_p2 = 0

# Dictionnaire des boutons
boutons = {}

# Fonction pour dessiner les symboles
def dessiner_symboles():
    """ Dessine les symboles (pions) sur la grille. """
    for i in range(3):
        for j in range(3):
            
            x = j * TAILLE_CASE + TAILLE_CASE // 2
            y = i * TAILLE_CASE + TAILLE_CASE // 2
            if noeud_actuel.grille[i][j] == 1:
                pygame.draw.circle(screen, Noir, (x, y), TAILLE_CASE // 4)
            elif noeud_actuel.grille[i][j] == -1:
                pygame.draw.circle(screen, Gris, (x, y), TAILLE_CASE // 4)

# Vérifie si les pions sont bien placés
print(noeud_actuel.grille)  # Ajoute ceci pour déboguer

# Fonction pour vérifier le gagnant
def verifier_gagnant():
    return noeud_actuel.check_winner()

# Fonction pour vérifier si la grille est pleine
def verifier_grille_pleine():
    return noeud_actuel.est_pleine()

# Boucle principale du jeu
running = True
mouvements = []
case_origine = None

while running:
    # Afficher l'image de fond
    screen.blit(image_fond, (0, 0))

    # Dessiner les symboles sous forme de boutons
    dessiner_symboles()
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.MOUSEBUTTONDOWN:
            x, y = pygame.mouse.get_pos()
            i, j = y // TAILLE_CASE, x // TAILLE_CASE

            if noeud_actuel.grille[i][j] == joueur_actuel:
                case_origine = (i, j)
                mouvements = noeud_actuel.deplacements_possibles(i, j)

        elif event.type == pygame.MOUSEBUTTONUP:
            if case_origine is not None:
                x, y = pygame.mouse.get_pos()
                i, j = y // TAILLE_CASE, x // TAILLE_CASE

                if (i, j) in mouvements:
                    noeud_actuel.grille[i][j] = joueur_actuel  # Déplacer le pion
                    noeud_actuel.grille[case_origine[0]][case_origine[1]] = 0  # Vider l'ancienne case
                    joueur_actuel = -joueur_actuel  # Changer de joueur
                
#                 #implémentation de minimax
#                 # Si c'est le tour du joueur gris (l'ordinateur)
#                     if joueur_actuel == -1:
#                         # Appel de la fonction minimax pour obtenir le meilleur coup
#                         _, meilleur_coup = minimax(noeud_actuel, depth=3, us=-1)
#                         if meilleur_coup:
#                             i, j = meilleur_coup  # Coordonnées du coup choisi
#                             noeud_actuel.grille[i][j] = -1  # Joueur gris effectue le coup
#                             joueur_actuel = 1  # Changement de joueur
                            
                case_origine = None
                mouvements = []



    # Mettre à jour l'affichage
    pygame.display.flip()
    
# Vérifier le gagnant
gagnant = verifier_gagnant()
if gagnant:
    print(f"Le gagnant est : {gagnant}")
elif verifier_grille_pleine():
    print("Match nul !")

# Quitter Pygame
pygame.quit()



pygame 2.6.1 (SDL 2.28.4, Python 3.10.9)
Hello from the pygame community. https://www.pygame.org/contribute.html
[[1, 1, 1], [0, 0, -1], [-1, 0, -1]]


AttributeError: 'Noeud' object has no attribute 'est_pleine'